In [1]:
# Import Library
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import re


In [2]:
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2023-01-01,2023-12-21&runtime=1,&num_votes=1,&moviemeter=1,&countries=ID"

# Buat instance Chrome service dengan path ke ChromeDriver executable
chrome_service = Service(r'C:\Users\ASUS\(DGX)\Kursus Asisten\chromedriver-win64\chromedriver.exe')

driver = webdriver.Chrome(service=chrome_service)
driver.maximize_window()

# Buat instance WebDriverWait
wait = WebDriverWait(driver, 30)
wait2 = WebDriverWait(driver, 15)

data = []

# Jumlah maksimum scroll yang ingin dilakukan
max_scrolls = 4

total_scrolls = 0

driver.get(url)


for item in range(2):
    for i in range(19):
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body'))).send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)  # Tunggu sejenak untuk memberikan waktu halaman merespon

    time.sleep(3)
    wait2.until(EC.presence_of_element_located((By.CLASS_NAME, "ipc-see-more__text")))
    print("proses ke bawah selesai - div see more sudah ada")
    time.sleep(2)

    try:
        see_more_button = wait2.until(EC.visibility_of_element_located((By.CLASS_NAME, "ipc-see-more__button")))
        print("Button see more sudah ada")
        time.sleep(3)
        print("Button see more sudah bisa diklik")

        driver.execute_script("arguments[0].click();", see_more_button)
        
        total_scrolls += 1
        print("total scrolls : ", total_scrolls)
    except Exception as e:
        print("Error saat mencoba klik tombol 'See more':", str(e))
        break  # Keluar dari loop jika tidak dapat menemukan atau mengklik tombol
    
for i in range(21) :
    wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body'))).send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)  # Tunggu sejenak untuk memberikan waktu halaman merespon

# Dapatkan judul dan rating dari elemen-elemen yang baru muncul
titles = driver.find_elements(By.CLASS_NAME, "ipc-title-link-wrapper")
durations = driver.find_elements(By.CLASS_NAME, "sc-43986a27-8")
ratings = driver.find_elements(By.CLASS_NAME, "ratingGroup--imdb-rating")

movie_data = []


for index_movie, (title, rating) in enumerate(zip(titles, ratings), start=1):
    movie_data.append({
        "id": f"movie_{index_movie}",
        "title": title.text,
        "rating": rating.text
    })


duration_data = []
index = 1

for duration in durations:
    duration_text = duration.text

    # Pengecekan apakah teks durasi berisi "2023"
    if "2023" in duration_text:
        continue

    # Pengecekan apakah teks durasi mengandung "h" atau "m"
    elif any(symbol in duration_text for symbol in ["h", "m"]):
        duration_data.append({
            "id": f"movie_{index}",
            "duration": duration_text
        })
        index += 1  # Increment the index when a duration is found

    # Pengecekan apakah teks durasi mengandung kata tertentu
    elif any(keyword in duration_text for keyword in ["R", "TV-MA", "TV-14", "TV-PG", "TV-G", "Not Rated", "Unrated"]):
        continue
    
    else:
        duration_data.append({
            "id": f"movie_{index}",
            "duration": "N/A"
        })
        index += 1  # Increment the index when no duration is found
    
            
        

print("semua proses selesai")

driver.quit()


proses ke bawah selesai - div see more sudah ada
Button see more sudah ada
Button see more sudah bisa diklik
total scrolls :  1
proses ke bawah selesai - div see more sudah ada
Button see more sudah ada
Button see more sudah bisa diklik
total scrolls :  2
semua proses selesai


In [3]:
len(duration_data)

109

In [4]:
len(movie_data)

109

In [5]:
import pandas as pd

df_duration = pd.DataFrame(duration_data)
df_movie = pd.DataFrame(movie_data)

# Menggabungkan dua dataframe
result_df = pd.merge(df_movie, df_duration, on="id", how="inner")


In [6]:
result_df

,id,title,rating,duration
0,movie_1,1. Operation Fortune: Ruse de Guerre,6.3\n (78K),1h 54m
1,movie_2,2. Suzzanna: Kliwon Friday Night,5.8\n (313),2h 12m
2,movie_3,3. Siksa Neraka,5.6\n (103),1h 38m
3,movie_4,4. Jatuh Cinta Seperti di Film-Film,8.8\n (373),1h 58m
4,movie_5,5. 172 Days,6.8\n (214),1h 43m
...,...,...,...,...
104,movie_105,105. Surga di Bawah Langit,6.9\n (16),1h 29m
105,movie_106,106. The Heart of Suhita,6.7\n (140),2h 17m
106,movie_107,107. Bismillah Kunikahi Suamimu,5.3\n (380),1h 50m
107,movie_108,108. Cherish & Ruelle,9.5\n (53),1h 12m


In [8]:
# Convert ke csv
result_df.to_csv('dataset/imdb_id_movie_2023.csv')